# Main function

In [2]:
import requests
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup as bs4
import re
import pandas as pd
import numpy as np
import json

class MOPS_ALL_2018(object):
    
    def __init__(self, sid, year, season, rid):
        self.sid=sid
        self.year=year
        self.season=season
        self.rid=rid
        keyword={'sid':self.sid, 'y':self.year, 's':self.season, 'rid':self.rid}
        self.header= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
        self.url='https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={sid}&SYEAR={y}&SSEASON={s}&REPORT_ID={rid}'.format(**keyword)
        self.get_tbl()
        
    def fetchall(self):
        self.fetch_BS()
        self.fetch_PLS()
        self.fetch_CFS()
        #self.fetch_SES()
        self.jsondata = {'data': [self.Balance, self.ProfitLoss , self.CashFlows]}
        
        return self.jsondata        
    
    def get_tbl(self):
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        res = web_ss.get(url=self.url, headers=self.header, timeout=5)
        res.encoding = 'big5'
        try:
            if res.status_code == 200:
                #soup = bs4(res.text, "lxml")
                soup = bs4(res.text, "lxml")
                self.tables = soup.select('table')
        except Exception as e:
            print(e)
            
        return self.tables
            
            
    def fetch_BS(self, Ntable=1):
        tr_list = self.tables[Ntable].select('tr')
        
        Balance_list = []
        for tr in tr_list:
            td_list = tr.select("td")
            if len(td_list) != 0:
                row = []
                for i in td_list:
                    row.append(i.text.strip().replace(',',''))
        #             print(i.text.strip().replace(',',''))
        #     print(tr.select("td"))
                if len(row) > 0:
                    Balance_list.append(row[:2])
          
#         self.Balance = pd.DataFrame(tbl, columns = ['cht','value'])
#         self.Balance = self.Balance[self.Balance['value'] != '']  
        
        self.Balance={'Balance':Balance_list}

        return self.Balance
        
    def fetch_PLS(self, Ntable=2):        
        tr_list = self.tables[Ntable].select('tr')
        
        ProfitLoss_list = []
        for tr in tr_list:
            td_list = tr.select("td")
            if len(td_list) != 0:
                row = []
                for i in td_list:
                    row.append(i.text.strip().replace(',',''))
        #             print(i.text.strip().replace(',',''))
        #     print(tr.select("td"))
                if len(row) > 0:
                    ProfitLoss_list.append(row[:2])
        
#         self.ProfitLoss = pd.DataFrame(tbl, columns = ['cht','value'])
#         self.ProfitLoss = self.ProfitLoss[self.ProfitLoss['value'] != '']  
        self.ProfitLoss={'ProfitLoss':ProfitLoss_list}
        return self.ProfitLoss
    
    
        
    def fetch_CFS(self, Ntable = 3):
        tr_list = self.tables[Ntable].select('tr')
        
        CashFlows_list = []
        for tr in tr_list:
            td_list = tr.select("td")
            if len(td_list) != 0:
                row = []
                for i in td_list:
                    row.append(i.text.strip().replace(',',''))
        #             print(i.text.strip().replace(',',''))
        #     print(tr.select("td"))
                if len(row) > 0:
                    CashFlows_list.append(row[:2])
        
#         self.CashFlows = pd.DataFrame(tbl, columns = ['cht','value'])
#         self.CashFlows = self.CashFlows[self.CashFlows['value'] != '']  
         
        self.CashFlows={'CashFlows':CashFlows_list}
        return self.CashFlows
        
        
    def fetch_SES(self, Ntable = 4):

        Etable = self.tables[Ntable]
        #print(Etable)
#         print(len(Etable.select("tr")))
        
        tr_list = Etable.select("tr")
        tbl = []
        column = []
        
        for tr in tr_list:
            td_list = tr.select("td") #row
            th_list = tr.select("th") #column
        
            row = []
            for td in td_list:
                row.append(td.text.strip().replace("\n","").replace(",",""))
            if len(row) > 0:
                tbl.append(row)
            
            for th in th_list:
                column.append(th.text.strip().replace("\n",""))
#         print(len(tbl))
#         print(column)
        self.equity_dict = {'col_index':column,'row_value':tbl}
#         print(self.equity_dict)
        return self.equity_dict
        
#         self.equity = pd.DataFrame(tbl,columns = column)


#         self.equity = self.equity.replace("", np.nan)
#         self.equity = self.equity.fillna(0)
#         self.equity['items'] = self.equity.iloc[:,0]
#         self.equity = self.equity.iloc[:,1:]
        
#         self.equity.set_index('items', inplace=True) 
#         return self.equity
#         return print(tbl)

In [3]:
class MOPS_ALL_2018_SES(object):
    
    def __init__(self, sid, year, season, rid):
        self.sid=sid
        self.year=year
        self.season=season
        self.rid=rid
        keyword={'sid':self.sid, 'y':self.year, 's':self.season, 'rid':self.rid}
        self.header= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}
        self.url='https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={sid}&SYEAR={y}&SSEASON={s}&REPORT_ID={rid}'.format(**keyword)
        self.get_tbl()
        self.fetch_SES()
          
    def get_tbl(self):
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        res = web_ss.get(url=self.url, headers=self.header, timeout=5)
        res.encoding = 'big5'
        try:
            if res.status_code == 200:
                #soup = bs4(res.text, "lxml")
                soup = bs4(res.text, "lxml")
                self.tables = soup.select('table')
        except Exception as e:
            print(e)
            
        return self.tables
    
    def fetch_SES(self, Ntable = 4):

        Etable = self.tables[Ntable]
        #print(Etable)
#         print(len(Etable.select("tr")))

        tr_list = Etable.select("tr")
        tbl = []
        column = []

        for tr in tr_list:
            td_list = tr.select("td") #row
            th_list = tr.select("th") #column

            row = []
            for td in td_list:
#                 print(row)
                row.append(td.text.strip().replace("\n","").replace(",",""))
            if len(row) > 0:
                tbl.append(row)

            for th in th_list:
#                 print(th)
                column.append(th.text.strip().replace("\n",""))
#         print(len(tbl))
#         print(column)
        self.equity_dict = {'col_index':column,'row_value':tbl}
        return self.equity_dict

In [50]:
code = '1201'
year='2018'
season='3'
rid = 'C'
stock_test = MOPS_ALL_2018_SES(code,year,season,rid)

data = stock_test.fetch_SES()
print(data)

{'col_index': ['', '普通股股本', '股本合計', '資本公積', '法定盈餘公積', '未分配盈餘（或待彌補虧損）', '保留盈餘合計', '國外營運機構財務報表換算之兌換差額', '備供出售金融資產未實現利益（損失）', '其他權益項目合計', '歸屬於母公司業主之權益總計', '非控制權益', '權益總額'], 'row_value': [['期初餘額', '5060629', '5060629', '688', '475607', '-216771', '258836', '-149698', '3875', '-145823', '5174330', '2633586', '7807916'], ['追溯適用及追溯重編之影響數', '0', '0', '0', '0', '3875', '3875', '0', '-3875', '-3875', '0', '0', '0'], ['期初重編後餘額', '5060629', '5060629', '688', '475607', '-212896', '262711', '-149698', '0', '-149698', '5174330', '2633586', '7807916'], ['本期淨利（淨損）', '0', '0', '0', '0', '1020605', '1020605', '0', '0', '0', '1020605', '-113225', '907380'], ['本期其他綜合損益', '0', '0', '0', '0', '0', '0', '-58281', '0', '-58281', '-58281', '-223', '-58504'], ['本期綜合損益總額', '0', '0', '0', '0', '1020605', '1020605', '-58281', '0', '-58281', '962324', '-113448', '848876'], ['非控制權益增減', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '-2507992', '-2507992'], ['權益增加(減少)數', '0', '0', '0', '0', '1020605', '1020605', '-

# Fetch DATA
#####  bl_cf_pl 2018_Q1還沒抓完

In [ ]:
import os
import time
import random
from requests.exceptions import ConnectionError
from codes import codes


def dlcheck(path):
    filename=os.listdir(path)
    code_cap=[]
    for _file in filename:
        code_cap.append(_file[0:4])
    return code_cap

year='2017'
season='3'
path=year+'Q'+season
csvpath=year+'Q'+season+'SES'

if not os.path.exists(path):
    os.mkdir(path)

if not os.path.exists(csvpath):
    os.mkdir(csvpath)

#取得兩個資料夾的交集，以便確認是否抓過
code_cap=set(dlcheck(path)) & set(dlcheck(csvpath))
print(code_cap)


for code,v in codes.items():
    if v.type=="股票" and v.market=="上市":
        try:
            if code not in code_cap:
                rid ='C'           
                stock=MOPS_ALL_2018(code,year,season,rid)
#                 print(stock.tables)
                if len(stock.tables) < 3:
                    time.sleep(random.uniform(2,5))
                    rid = 'A'
                    stock=MOPS_ALL_2018(code,year,season,rid)
                    if len(stock.tables) < 3:
                        with open('nofinreport.txt', 'a') as f:
                            f.write('nodata:'+code+':'+stock.url+'\n')
                        print(stock.url)
                        print(code +' no finance data!')
                        continue
                    
                if stock.tables != []:
                    print('fetch BS, PLS and CFS from:' ,stock.url)
                    data = stock.fetchall()
                    filename=path+'/'+code+'-'+year+'-'+'Q'+season+'.json'
                    with open(filename, 'w', encoding='utf8') as f:
                        json.dump(data, f)
                    
                    
                    print('fetch SES from:' ,stock.url)

                    jsonname=csvpath+'/'+code+'-'+year+'-'+'Q'+season+'_ses.json'
                    ses_data = stock.fetch_SES()
                    with open(jsonname, 'w', encoding='utf8') as f:
                        json.dump(ses_data, f)
                        
                        
                else:
                    print('error-nodata:', code) 
                    with open('error.log', 'a+') as f:   
                        f.write('nodata:'+ code+':' + stock.url+'\n\n')
                time.sleep(random.uniform(2,5))             
                
        except ConnectionError:
            code_cap=set(dlcheck(path)) & set(dlcheck(csvpath))
            time.sleep(120)
            continue
print('download finished!')

set()
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1102&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1102&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1103&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1103&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1104&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1104&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.t

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1313&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1313&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1314&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1314&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1315&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1315&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1316&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1316&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1443&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1443&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1444&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1444&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1445&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1445&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1446&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1446&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1514&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1514&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1515&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1515&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1516&SYEAR=2017&SSEASON=3&REPORT_ID=A
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1516&SYEAR=2017&SSEASON=3&REPORT_ID=A
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1517&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1517&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1604&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1604&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1605&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1605&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1608&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1608&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1609&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1609&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

...

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6024&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6024&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6108&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6108&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6112&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6112&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6206&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6206&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6209&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6209&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6213&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6213&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6214&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6214&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6451&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6451&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6452&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6452&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6456&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6456&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6464&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6464&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.co

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8072&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8072&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8081&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8081&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8101&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8101&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8103&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8103&SYEAR=2017&SSEASON=3&REPORT_ID=C
https://mops.twse.com.tw/server-java/t164sb01?st

fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8473&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8473&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8478&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8478&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8480&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8480&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch BS, PLS and CFS from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8481&SYEAR=2017&SSEASON=3&REPORT_ID=C
fetch SES from: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=8481&SYEAR=2017&SSEASON=3&REPORT_ID=C
https://mops.twse.com.tw/server-java/t164sb01?st